# Single Subject Segmentation Q1K

In [ ]:
# import packages
#import q1k_sync_tools as qst
import mne
import mne_bids
#mne.viz.set_browser_backend('qt')
#%matplotlib qt
#pd.options.mode.chained_assignment = None
#import pylossless as ll
#import numpy as np
import pandas as pd
#import glob

from pathlib import Path
import shutil
import pylossless as ll


In [ ]:
project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
pylossless_path = "derivatives/pylossless/"
postproc_path = "derivatives/q1k_postproc/"
task_id_in = "mn"
task_id_out = "mn"
subject_id = '014'
session_id = '02'

bids_path = mne_bids.BIDSPath(
    subject=subject_id, session=session_id, task=task_id_out, run="1", datatype="eeg", suffix="eeg",root=project_path + pylossless_path
)
print(bids_path)

In [ ]:
bids_path

In [ ]:
# Read the BIDS pylossless output file..
eeg_raw = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False)

In [ ]:
# Get the events form the annotations
eeg_events, eeg_event_dict  = mne.events_from_annotations(eeg_raw)

In [ ]:
ll_state = ll.LosslessPipeline()
ll_state = ll_state.load_ll_derivative(bids_path)


In [ ]:
bids_path_str=str(bids_path)
# Merge marks down to bads (aka manual)
ll_qcr = ll_state.raw.copy()
manual = []
for flag_type in ll_state.flags['ch']:
    manual.extend(ll_state.flags['ch'][flag_type])
print(ll_state.flags['ch'])
ll_qcr.info['bads'].extend(manual)
ll_qcr.info['bads'] = list(set(ll_qcr.info['bads']))

fig = ll_qcr.plot_sensors(show_names=True)

# Read the ICLabel info from file and add to exclude
df = pd.read_csv(bids_path_str.replace('_eeg.edf', '_iclabels.tsv'), sep='\t')
ll_state.ica2.exclude = list(df[df['ic_type'].str.match('eog|muscle|ch_noise|ecg')].index)

# Load the data and apply the ICA
ll_qcr.load_data()
ll_state.ica2.apply(ll_qcr)
ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=40.0)
ll_qcr = ll_qcr.interpolate_bads()
ll_qcr = ll_qcr.set_eeg_reference(ref_channels="average")
ll_qcr

In [ ]:
# Relabel condition vars for niceness
eeg_event_dict['mmn/standard/disp/mmns'] = eeg_event_dict.pop('mmns')
eeg_event_dict['mmn/target/disp/mmnt'] = eeg_event_dict.pop('mmnt')

In [ ]:
# Epoch structure is created for ALL events, then you 'segment' by indexing into it
epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1, tmax=2.0, on_missing='warn', event_repeated='drop')
epochs[['mmns', 'mmnt']]
epochs.save(project_path + pylossless_path + postproc_path + 'epoch_fif_files/mn/' + bids_path.basename + '_epochs.fif', overwrite=True)

In [ ]:
# Plot ERPs
evokeds = {'mmns': epochs['mmns'].average(), 'mmnt': epochs['mmnt'].average()}
mne.viz.plot_compare_evokeds(evokeds, picks=['E6'], combine='mean')

In [ ]:
mne.write_evokeds(project_path + pylossless_path + postproc_path + 'erp_fif_files/mn/' + bids_path.basename + '_erp.fif',list(evokeds.values()), overwrite=True)